In [ ]:
!pip install nltk
!pip install gensim
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import wordnet as wn
wn.lemmas('break', pos='n') # Retrieve all lexemes for the noun 'break'
l1 = wn.lemmas('break', pos='n')[0]
s1 = l1.synset() # get the synset for the first lexeme
s1
#s1.lemmas()
#s1.lemmas()[0].name() # Get the word of the first lexeme
#s1.definition()
#s1.examples()
#s1.hypernyms()
#s1.hyponyms()
#l1.count()

Synset('interruption.n.02')

In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2021-06-12 01:15:05--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.224.99
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.224.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  88.9MB/s    in 17s     

2021-06-12 01:15:22 (93.9 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True) 

In [ ]:
v1 = model.wv['computer']
#v1 = model.get_vector('computer)
v1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [ ]:
len(v1)

300

In [ ]:
model.similarity('computer','calculator')
#model.similarity('computer','toaster')
#model.similarity('computer','dog')
#model.similarity('computer','run')

0.3339888

In [ ]:
!pip install transformers

In [ ]:
#!python lexsub_xml.py lexsub_trial.xml
#!head smurf.predict # print the first 10 lines of the file
#!perl score.pl smurf.predict gold.trial

In [ ]:
# lexsub_main.py

#!/usr/bin/env python
import sys

from lexsub_xml import read_lexsub_xml
from lexsub_xml import Context 

# suggested imports 
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

import numpy as np
import tensorflow as tf

import gensim
import transformers 

import string

from typing import List

def tokenize(s): 
    """
    a naive tokenizer that splits on punctuation and whitespaces.  
    """
    s = "".join(" " if x in string.punctuation else x for x in s.lower())    
    return s.split() 

def get_candidates(lemma, pos) -> List[str]:
    # Part 1
    # pos is 'a','n','v','r' for adj, noun, verb, adv
    # Takes a lemma and POS and returns set of possible substitutes

    substitutes = []
    lemmas = wn.lemmas(lemma, pos)

    # Look up the lemma and POS in WordNet and get all synsets that the lemma appears in
    for lem in lemmas:
      synset = lem.synset()

      # Then, obtain all lemmas that appear in any of these synsets
      for candidate in synset.lemmas():
        candidate_name = candidate.name()
        # Make sure output doesn't contain the input lemma itself
        if candidate_name == lemma:
          continue
        else:
          # Remove the '_' in 'turn_around' for ex and add to list
          substitutes.append(candidate_name.replace("_",""))
      
    # Return set of possible substitutes
    substitutes = set(substitutes)
    return substitutes #replace for part 1

def smurf_predictor(context : Context) -> str:
    """
    suggest 'smurf' as a substitute for all words.
    """
    return 'smurf'

def wn_frequency_predictor(context : Context) -> str:
    # Takes a context object as input and predicts the possible synonym w highest total 
    # occurance freq according to wordnet

    # Sum up the occurance counts for all senses of the word if the word and target appear together 
    # in multiple synsets

    # Duplicated code from the get_candidates method for finding candidate synonyms
    occurence_freqs = {}
    lemmas = wn.lemmas(context.lemma, context.pos)
    for lem in lemmas:
      synset = lem.synset()
      for candidate in synset.lemmas():
        # Make sure output doesn't contain the input lemma itself
        if candidate.name() == context.lemma:
          continue
        else:
          # Remove the '_' in 'turn_around' for ex and add to list
          candidate = candidate.name().replace('_', ' ')
          # freqs of lemma according to wordnet
          if occurence_freqs[candidate] not in occurence_freqs:
            occurence_freqs[candidate] = candidate.count()
          else:
            occurence_freqs[candidate] += candidate.count()

    # highest total occurence freq according to wordnet
    # max(iterable, key=dict.get)
    pred = max(occurence_freqs, key = occurence_freqs.get)

    # Should give ~10% precision and recall
    return pred # replace for part 2

def wn_simple_lesk_predictor(context : Context) -> str:
    # Select a synset for the target word using WSD
    # To perform WSD, implement the Lesk alg
   
   # You may want to remove stopwords
    stop_words = stopwords.words('english')
    overlap = {}
    
    # Look at all possible synsets that the target word appears in
    for synset in wn.lemmas(context.lemma, context.pos):
      lemma = synset.synset()
      def_ = lemma.definition()
      #print(def_)

      if len(wn.lemmas(context.lemma,context.pos)) != 1:
        # Add to the def: all exs for the synset + the def and all exs for all hypernyms of the snyset
        # all exs for synset:
        for elt in lemma.examples():
          def_= def_+" "+elt
        # all exs of hypernyms
        for elt in lemma.hypernyms():
          for elem in elt.examples():
            def_=def_+" "+elt.definition()+" "+elem
        
        # remove stopwords
        # As a subtask, tokenize and normalize the defs and exs in wordnet by either
        # looking up various tokenization methods in NLTK or using the tokenize method provided w the code
        
        #def_.split()
        #print(def_)

        def_=tokenize(def_)
        cleaned_def=[]
        for elt in def_:
          if elt in stop_words:
            continue
          else:
            cleaned_def.append(elt)
        
        target_sen = context.left_context + context.right_context
        #print(target_sen)

        #target_sen = tokenize(target_sen)
        cleaned_sen = []
        for elt in target_sen:
          if elt in stop_words:
            continue
          else:
            cleaned_sen.append(elt)
        
        #print(cleaned_def)
        #print(cleaned_sen)
        # Compute the overlap between the def of the synset and the context of the target word
        cleaned_def = set(cleaned_def)
        cleaned_sen = set(cleaned_sen)
        overlap[synset] = (len(cleaned_def.intersection(cleaned_sen)))
        #print(overlap)
       
    # sort dict by values from largest to smallest
    sorted_overlap = dict(sorted(overlap.items(), key = lambda x:x[1], reverse = True))
    #print(sorted_overlap)

    if sorted_overlap:
      max_overlap = max(sorted_overlap.values())
    else:
      max_overlap = 0

    keys = []
    for key, value in overlap.items():
      if max_overlap== sorted_overlap:
        # If there is not overlap (ie. tie), select the most frequent synset
        # ie the synset with which the target word forms the most frequent lexeme
        continue
      else:
        if value == max_overlap:
          keys.append(key)

    # Then, select the most frequent lexeme from that synset as the result
    
    #max_counts = -1
    #count = 0
    counts = {}
    for synset in wn.lemmas(context.lemma, context.pos): 
      if synset in keys:
        for elt in synset.synset().lemmas():
          #count+=elt.count()
          counts[elt.name()] = elt.count()
          #if max_count<count:
            #most_freq_syn = synset
            #max_count = count

    # Return the most frequent synonym from that synset as a substitute
    counts = dict(sorted(counts.items(),key=lambda x:x[1], reverse=True))
    for syn in list(counts.keys()):
      if syn != context.lemma:
        return syn
        #return elt.replace("_","") # replace for part 3

class Word2VecSubst(object):
        
    def __init__(self, filename):
        self.model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=True)    

    def predict_nearest(self,context : Context) -> str:
        # First obtain a set of possible synonyms from wordnet either by using the method from part 1 or rewriting the code
        possible_syns = get_candidates(context.lemma, context.pos)
        
        max_similarity = self.model.similarity(possible_syns[0],context.lemma)

        # Return the synonym that is most similar to the target word, according to the word2vec embeddings
        for i in range(1,len(possible_syns)):
          old_max_similarity = max_similarity
          max_similarity = max(old_max_similarity, self.model.similarity(possible_syns[i],context.lemma))
          if max_similarity == old_max_similarity:
            synonym = possible_syns[i]
          else:
            synonym = possible_syns[i-1]

        # Precision and recall should be ~11%
        return synonym #replace for part 4
  
    def best_predictor(self, context : Context) -> str: 
      #PART 6 HERE, can try to calculate a weighted similarity sum 
      d = {}
      #stop_words = stopwords.words('english') 

      # obtain a set of candidate syns using get_candidates
      candidate_syns = get_candidates(context.lemma, context.pos)
          
      w1 = 0.5
      w2 = 0.5
      w3 = 0.5
      for syn in candidate_syns:
        try:
          output1 = self.model.similarity(syn, context.lemma)
          output2 = self.model.similarity(syn, context.left_context[-1])
          output3 = self.model.similarity(syn, context.right_context[0])
        except KeyError:
          continue
          
        if syn in d:
          d[syn] += w1*output1
          d[syn] += w2*output2
          d[syn] += w3*output3
        else:
          d[syn] = w1*output1
          d[syn] += w2*output2
          d[syn] += w3*output3
        
      high_low_d = dict(sorted(d.items(), key=lambda x:x[1], reverse=True))

      return list(high_low_d.keys())#[0]


class BertPredictor(object):

    def __init__(self): 
        self.tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.model = transformers.TFDistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

    def predict(self, context : Context) -> str:
        # feed the context sentence into BERT and it will tell us good replacements for the target word
        # output vector is length |V| for each word, coontains score for each

        # obtain a set of candidate syns using get_candidates
        candidate_syns = get_candidates(context.lemma, context.pos)
        
        # Convert the info in context into a suitable masked input repr for the DisttilBERT model
        # DistilBERT: feed the context sentence into BERT and it will tell us good replacements for the target word
        # need: context sentence
        context_sentence = ""
        for left in context.left_context:
          is_str = left.isalpha()
          if not is_str:
            context_sentence += left
          else:
            context_sentence = context_sentence + " " + left
        context_sentence = context_sentence + " " + "[MASK]"

        for right in context.right_context:
          is_str = right.isalpha()
          if not is_str:
            context_sentence += right
          else:
            context_sentence = context_sentence + " " + right

        # convert sentence to list
        li = list(context_sentence.split(" "))
        #print(li)
        # store the index of the masked target word
        masked_idx = li.index("[MASK]")

        #print(masked_idx)

        # Run the DistilBERT model on the input repr
        # output vector is length |V| for each word, coontains score for each
        output = self.model.predict(np.array(li).reshape((1, -1))
     
        # Select from the set of wordnet derived candidate syns the highest-scoring word in target position
        # ie. the position of the masked word
        word = output[masked_idx]
        return word

    
if __name__=="__main__":

    # At submission time, this program should run your best predictor (part 6).

    #W2VMODEL_FILENAME = 'GoogleNews-vectors-negative300.bin.gz'
    #predictor = Word2VecSubst(W2VMODEL_FILENAME)
    #predictor = BertPredictor()

    for context in read_lexsub_xml("lexsub_trial.xml"):#sys.argv[1]):
        #print(context)  # useful for debugging
        #prediction = smurf_predictor(context) 
        prediction = predictor.best_predictor(context)
        #prediction = wn_simple_lesk_predictor(context)
        #prediction = predictor.predict(context)
        print("{}.{} {} :: {}".format(context.lemma, context.pos, context.cid, prediction))


SyntaxError: ignored